In [1]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import date, timedelta, datetime
import sqlite3    
import re   
import requests   
from bs4 import BeautifulSoup   
import plotly.graph_objects as go
#import win32com.client as win32
import plotly.io as pio
from plotly.subplots import make_subplots

In [2]:
pd.set_option('display.max_rows', None)

#### Funções Data

In [3]:
def day_n (offset):         
    
    return datetime.today()-timedelta(days = offset)         

In [4]:
def sunday(offset, day):    
    
    if day.weekday()==6:     
        day = day_n(offset+2)         
        if day.strftime("%d-%m-%Y") == holiday(offset, day, dataf).strftime("%d-%m-%Y"):         
            return day         
        else:              return holiday(offset, day, dataf)         
    
    return day   

####  Caso seja domingo essa função retorna o última dia antes de começar o final de semana, seja ele feriado ou não.  ####  Caso não seja domingo ele retorna o mesmo dia que foi passado como argumento pela função, sem alterá-lo.   

In [5]:
def saturday(offset, day):
    
    if day.weekday()==5:     
        day = day_n(offset+1)         
        if day.strftime("%d-%m-%Y") == holiday(offset, day, dataf).strftime("%d-%m-%Y"):         
            return day         
        else:              return holiday(offset, day, dataf)     
    return day   

####  Caso seja sabado essa função retorna o última dia antes de começar o final de semana, 
####  seja ele feriado ou não.  
####  Caso não seja sábado ele retorna o mesmo dia que foi passado como argumento pela função, sem alterá-lo.   

In [6]:
def holiday(offset, day, dataf):     
    
    for i in dataf:     
        if (i[8:]+"-"+i[5:7]+"-"+i[0:4])==day.strftime("%d-%m-%Y"):             
            return holiday(offset+1, day_n(offset+1), dataf)         
    
            
    if day.strftime("%d-%m-%Y") == sunday(offset, day).strftime("%d-%m-%Y"):         
    
        
        if day.strftime("%d-%m-%Y") == saturday(offset, day).strftime("%d-%m-%Y"):             
            
            return day         
        else:             
            return saturday(offset, day)         
    else:         
        return sunday(offset, day)         

In [7]:
##### Descobrindo o D-1 #####

def find_last_workday( offset, day, dataf): 
    
    if day.strftime("%d-%m-%Y") == holiday(offset, day, dataf).strftime("%d-%m-%Y"):         
        
        if day.strftime("%d-%m-%Y") == sunday(offset, day).strftime("%d-%m-%Y"):             
            
            if day.strftime("%d-%m-%Y") == saturday(offset, day).strftime("%d-%m-%Y"):                 
                return day                 
                
            else:                     
                return saturday(offset, day)         
        else:                 
            return sunday(offset, day)         
    else:         
        return holiday(offset, day, dataf)          

##### Vetor de feriados

In [8]:
def holidays():
    
    feriados = pd.read_csv(r"C:\Users\psilv\Python\feriados.csv")   
 
 
    data = feriados['data'].tolist()   
 
    ########### Cria vetor de feriados 
    
    dataf=[] 
    
    for i in data:   
        
        dataf.append(i.strip()[-4:] + "-" + ("0"+ i.strip()[0] if i.find("/")==1 else i.strip()[0:i.find("/")]) + "-" + ("0" + re.search("/(.*?)/", i.strip()).group(1) if len(re.search("/(.*?)/", i.strip()).group(1))==1 else re.search("/(.*?)/", i.strip()).group(1)))
        
    return dataf     

In [9]:
def past_n_days(n):     
    day = datetime.today()-timedelta(days = n)     
    print("O último dia corrido é: " +  day.strftime("%d-%m-%Y"))     
    return day   

#####  Função auxiliar para criar Vetor de dias úteis 

In [10]:
##### Date Format input  YY-MM-dd,  run_days são números inteiros

##### O input de data dentro da função pandas.date_range deve ser no formato YY-MM-dd

def date_array (last_day, run_days):

   
    Datas = pd.date_range(end = last_day, periods = run_days).to_pydatetime().tolist()

    
    datas=[] 

    
    for k in range(len(Datas)):

        datas.append(Datas[len(Datas) - k -1])



 
    return datas

#### Função iterativa para criar Vetor de dias úteis.

In [11]:
#### Função para obter um vetor de dias úteis de um tamanho específico, por ser iterativa é custosa computacionalmente

#### Date format input dd-MM-YY, length é número inteiro


def work_days_array_length(last_day, length):


    i=length

    while len(date_array(last_day, i))!= length:

        if len(date_array(last_day, i))==length:

            break

        i+=1

    return date_array(last_day, i)

In [12]:
####### Criando vetor desde o ultimo dia do banco até o D-1

####### Start é a mais antiga, e End a mais recente

####### Date Format input  YY-MM-dd (SQL)


def  work_days_array_btw_dates(start_date, end_date):
    

    end = date(int(end_date[:4]), int(end_date[5:7]), int(end_date[-2:]))

    start = date(int(start_date[:4]), int(start_date[5:7]), int(start_date[-2:]))

    
    dif = end-start
    

    dif = int(abs(dif.days))
    
    
    print("Números de dias a serem atualizados:  " + str(dif))
    print("\n")
    
    #### No caso start_date=end_date é preciso desse if 
    
    if dif == 0:
        dif=2
            


    dates = date_array(end_date, 2*dif)

    aux=0
    

    for i in dates:

        if i.strftime("%Y-%m-%d")==start_date:

            aux = dates.index(i)
                 
    
    return dates[:aux+1]

#### Função para deletar linha específica

In [13]:
def delete_row_sql(table_name, column_name, conn, desired_info):
    
    
    query = ("""DELETE""" + 
             """ FROM """ + table_name +
             """ WHERE """ + column_name +
             """ = '""" + desired_info +
             """'; """)
    
    conn.execute(query)
    
    print("Deleted as Follows:")
    print("\n")
    print(query)

#### Função para obter um dataframe a partir de uma query do banco de dados

In [14]:
### Criar função para obter um dataframe entre datas específicas a partir de uma query em SQL

### MELHORAR PRA RANGE DE DATAS


def df_from_sql_query(table_name, date_column_name, conn, start_date, end_date):
    
    
    query = ("""SELECT * """ + 
             """ FROM """ + table_name +
             """ WHERE """ + date_column_name +
             """ BETWEEN strftime('%Y-%m-%d %H:%M:%S', '""" + start_date +
             """') AND strftime('%Y-%m-%d %H:%M:%S', '""" + end_date +
             """') ORDER BY """ + date_column_name + """; """)
    
    #print(query)

    df = pd.read_sql_query(query, conn)
    
    
    return df   

In [15]:
# df_from_sql_query('bitcoin', "Date", conn, "2022-01-28", "2022-07-25")

#### Inserindo novas linhas no banco de dados

In [16]:
##### Tornar essa função genérica para qualquer dataframe

def insert_n_lines_database(cursor, df, table_name, date_column_name):
    
           
    print("\n")     
    print("Os dados antes da atualização eram:")   
    print("\n")  
    
    for i in get_last_n_rows(cursor, table_name, 5, date_column_name):   
        print(i)             
 
    print("\n")     
   
    insert_n_rows(cursor, df, table_name)             
       
    print("\n")   
    print("Os novos dados são:")   
    print("\n")     
                   
    for i in get_last_n_rows(cursor, table_name, 5, date_column_name):   
        print(i)         
    print("\n")             
            

#### Atualizando as bases de dados

In [17]:
def updating_database(cursor, date_column_name):      
    
    for i in assets:
        
        print('\n')
        
        day = past_n_days(1)
    
        day = day.strftime("%Y-%m-%d")
        
        ###### Verificando a última data atualizada no banco de dados   
        last_date_db = get_last_db_date(cursor, date_column_name, i)[:10]        

        if last_date_db == day:         
            print("Os dados da cripto " + str(i) + " estão Atualizados!!")         
        else:         
            print("\n")     
            print("O último dia atualizado no banco de dados é: " + last_date_db)   
            print("\n")         

            #### Vetor de datas   
            dates = work_days_array_btw_dates(last_date_db, day)[:-1]   
            print("Datas a serem atualizadas no banco de dados:")         
            print("\n")         

            for j in range(len(dates)):             
                print(dates[j])             


            print("\n")
            print("Downloading from Yahoo Finance API ....")         
            print("\n")         

            #### Dataframe com os resultados
            
            print("Moeda Atualizada: " + assets[i])
    

            df = get_data_btw_date(assets[i],dates[-1:][0].strftime("%Y-%m-%d"), dates[0].strftime("%Y-%m-%d"))     


            #### Inserindo novos dados           
            insert_n_lines_database(cursor, df, i, date_column_name)  

            ##### Adicionar o insert lines dentro da função Web Scrap para atualizar o banco de dados em cada iteração e não
            ##### correr o risco de falhar o web scraping e perder os dados
    
    

#### Função para captar os dados da API do Yahoo Finance

In [39]:
#### Nome da moeda, exemplo: Ethereum é ETH-USD
#### Data format YY-MM-DD

def get_data_btw_date(crypto_name, start_date, end_date):
    
    crypto_df = yf.download(crypto_name, start= start_date, end= end_date)
    
    crypto_df = crypto_df.reset_index(level=0)
    
    print(crypto_df.dtypes)
    print(crypto_df)
    
    crypto_df['Date'] = pd.to_datetime(crypto_df['Date'], format='%Y-%m-%d %H:%M:%S')
    
    crypto_df['Date'] = crypto_df['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')

    print(crypto_df.dtypes)
    print(crypto_df)

    return crypto_df

In [19]:
a = get_data_btw_date('ADA-USD', '2022-08-21', '2022-08-21')

[*********************100%***********************]  1 of 1 completed
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object
        Date      Open      High       Low     Close  Adj Close     Volume
0 2022-08-21  0.451266  0.472278  0.448482  0.463546   0.463546  616272980
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object
        Date      Open      High       Low     Close  Adj Close     Volume
0 2022-08-21  0.451266  0.472278  0.448482  0.463546   0.463546  616272980


In [ ]:
print(a.dtypes)

#### Funções SQL

In [ ]:
## Insert crypto name in lower case

In [20]:
#### index can be False or True, to me worked as False

def create_table(df, name_desired, conn, index):
    
        df.to_sql(name_desired, conn, if_exists='fail', index=index)   
    


In [21]:
def rename_table(table_name, new_table_name, cursor):
    
    try:
        
        for i in get_names_tables(cursor):
        
            if new_table_name == i[0]:
                
                print("O Nome já está em uso!")
            
                raise Exception

        cursor.execute("""ALTER TABLE """ +  table_name + """ RENAME TO """ + new_table_name + """ ;""")

    except Exception:

        print("Não foi possível alterar o nome da tabela!")
        

In [22]:
def show_table(table_name, cursor):     
    
    c=cursor.execute("""SELECT * FROM """ + table_name ).fetchall()     
    for i in c:      
        print(i)     
    return c     

In [23]:
def delete_table(name, cursor):   
    
    cursor.execute("""DROP TABLE """ +  name ).fetchall()   

In [24]:
def last_database_day(crypto_name):

    sql_str = '''SELECT Date FROM ''' + crypto_name + ''' ORDER BY Date DESC LIMIT 1;'''

    c.execute(sql_str)

    date = c.fetchone()

    return date[0][:10]
    

In [25]:
def get_names_tables(cursor):     
    
    cursor.execute('''SELECT name FROM sqlite_master WHERE type='table';''')   
    
    return(cursor.fetchall())

In [26]:
def count_rows_table(cursor, table_name): 
    
    c = cursor.execute("""SELECT COUNT(*) FROM """ + table_name).fetchall()     
    return c[0][0]   

In [27]:
def get_columns_names(cursor, table_name): 
    
    c = cursor.execute("""PRAGMA table_info(""" + table_name + """);  """ ).fetchall()     
    col_names = []     
    for i in c:         
            col_names.append(i[1])
            
    return col_names   

In [28]:
def get_last_n_rows(cursor, table_name, n_rows, column_name):
    
    c = cursor.execute("""SELECT * FROM """ 
                       + table_name + 
                       """ ORDER BY """ +column_name + 
                       """ DESC LIMIT """+ str(n_rows) + 
                       """ ; """  ).fetchall()     
#     for i in c:      
#         print(i)


    
    return c     

In [29]:
def insert_n_rows(cursor, df, table_name):     
 
   
    new_values = []   
    
    for i in range(len(df)):    
        
        new_values.append(tuple(df.iloc[i]))         
    
    s=""     
    
    for i in new_values:   
        
        s+=str(i) + ","   
        
    
    s=s[:-1] 

    #print(s)
    
    
    cursor.execute("""INSERT INTO """ + table_name + """ VALUES """ + s + """ ; """  )     
    

In [30]:
def distinct_values(table_name, column_name, cursor):
    
    c = cursor.execute("""SELECT """ + column_name + """, COUNT(*) """ + """ FROM """ + table_name + """ GROUP BY """ +  column_name + """ ; """  ).fetchall()     
    
    
    for i in c:
        
        print(i)
    
       

In [31]:
def change_columns_names(cursor, table_name, column_old_name, column_new_name):   
 
    print("Old Columns Names: " + str(get_columns_names(cursor, table_name)) )     
    c = cursor.execute("""ALTER TABLE """ + table_name +                                 """ RENAME COLUMN """ + column_old_name +                                 """ TO """+column_new_name+                                 """ ; """  ).fetchall()     
    print("New Columns Names: " + str(get_columns_names(cursor, table_name)) )     
 

In [32]:
def get_last_db_date(cursor,column_name, table_name):  
    
    c=cursor.execute("""SELECT """ + column_name + 
                     """ FROM """ + table_name + 
                     """ ORDER BY """ + column_name + 
                     """ DESC LIMIT 1 ; """).fetchall()     
    return c[0][0]   

In [33]:
def database_info(cursor, table_name, date_column_name):
    
        
    
    print("O banco de dados contém as seguintes tabelas: ")
    
    print("\n")
    
    for i in get_names_tables(cursor):
        print(i[0])
    
    print("\n")
    
    print("O nome da tabela solicitada é :" + table_name)
    
    print("\n")
    
    print("O nome das colunas são: ")
    
    print("\n")
    
    print(get_columns_names(cursor, table_name))
    
    print("\n")
    
    print("O número total de linhas é: " + str(count_rows_table(cursor, table_name)))
    
    print("\n")
    
    print("A data mais recente dos dados é: " + str(get_last_db_date(cursor,date_column_name, table_name)))
    
    print("\n")
    
    print("Todas as datas distintas e quantas linhas cada uma possui são: ")
    
    print("\n")
    
    print(distinct_values(table_name, date_column_name, cursor)) 
    
    

In [34]:
def get_new_crypto_in_yf(crypto_name, table_title, conn, end_date, index=False):
    
 
    df = get_data_btw_date(crypto_name, '2000-01-01', end_date)
        
    #### Data escolhida é bem antes do surgimento das criptomoedas no planeta terra
        
        
    create_table(df, table_title, conn, index)
    

In [35]:
###### updating entire database for all assets ##########

def insert_new_criptos_in_db(assets):
    
    day = past_n_days(1)
    
    day = day.strftime("%Y-%m-%d")

    for i in assets:
        
        try:

            get_new_crypto_in_yf(assets[i], i, conn, day, index=False)
            
        
        except Exception:
            
            print(str(assets[i]+ " já  está no banco de dados!"))
            
            continue

In [36]:
assets = { 'BTC': 'BTC-USD', 
           'ETH': 'ETH-USD', 
           'USDT':'USDT-USD', 
           'USDC':'USDC-USD', 
           'BNB':'BNB-USD',
           'BUSD':'BUSD-USD',
           'XRP': 'XRP-USD',
           'ADA': 'ADA-USD',
           'SOL': 'SOL-USD',
           'DOGE':'DOGE-USD',
           'DOT': 'DOT-USD',
           'HEX': 'HEX-USD',
           'SHIB': 'SHIB-USD',
           'DAI': 'DAI-USD',
           'WTRX': 'WTRX-USD',
           'AVAX':'AVAX-USD',
           'MATIC': 'MATIC-USD',
           'TRX': 'TRX-USD',
           'STETH': 'STETH-USD',
           'WBTC': 'WBTC-USD',
           'UNI1': 'UNI1-USD',
           'LEO': 'LEO-USD',
           'ETC': 'ETC-USD',
           'LTC': 'LTC-USD'
         }

#### Rodando

In [40]:
conn = sqlite3.connect('crypto_data')
c = conn.cursor()

In [41]:
updating_database(c, "Date")



O último dia corrido é: 06-06-2023


O último dia atualizado no banco de dados é: 2022-08-19


Números de dias a serem atualizados:  291


Datas a serem atualizadas no banco de dados:


2023-06-06 00:00:00
2023-06-05 00:00:00
2023-06-04 00:00:00
2023-06-03 00:00:00
2023-06-02 00:00:00
2023-06-01 00:00:00
2023-05-31 00:00:00
2023-05-30 00:00:00
2023-05-29 00:00:00
2023-05-28 00:00:00
2023-05-27 00:00:00
2023-05-26 00:00:00
2023-05-25 00:00:00
2023-05-24 00:00:00
2023-05-23 00:00:00
2023-05-22 00:00:00
2023-05-21 00:00:00
2023-05-20 00:00:00
2023-05-19 00:00:00
2023-05-18 00:00:00
2023-05-17 00:00:00
2023-05-16 00:00:00
2023-05-15 00:00:00
2023-05-14 00:00:00
2023-05-13 00:00:00
2023-05-12 00:00:00
2023-05-11 00:00:00
2023-05-10 00:00:00
2023-05-09 00:00:00
2023-05-08 00:00:00
2023-05-07 00:00:00
2023-05-06 00:00:00
2023-05-05 00:00:00
2023-05-04 00:00:00
2023-05-03 00:00:00
2023-05-02 00:00:00
2023-05-01 00:00:00
2023-04-30 00:00:00
2023-04-29 00:00:00
2023-04-28 00:00:00
2023-04-27 0

[*********************100%***********************]  1 of 1 completed
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object
          Date         Open         High          Low        Close  \
0   2022-08-20  1612.650635  1652.218750  1534.298828  1577.003784   
1   2022-08-21  1576.543579  1640.936523  1569.046509  1619.318970   
2   2022-08-22  1619.166870  1622.778809  1535.014771  1622.505859   
3   2022-08-23  1622.939331  1666.676514  1569.428589  1662.769897   
4   2022-08-24  1662.695435  1686.557739  1610.099243  1657.059204   
5   2022-08-25  1657.336548  1718.183228  1656.856079  1696.457031   
6   2022-08-26  1696.324585  1698.561035  1498.771240  1507.782837   
7   2022-08-27  1508.156982  1517.150024  1454.282959  1491.395020   
8   2022-08-28  1491.206787  1505.791992  1430.547363  1430.547363   
9   2022-08-29  1430.

[*********************100%***********************]  1 of 1 completed
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object
          Date      Open      High       Low     Close  Adj Close  \
0   2022-08-20  1.000053  1.000133  0.999973  0.999974   0.999974   
1   2022-08-21  0.999974  1.000161  0.999942  0.999994   0.999994   
2   2022-08-22  0.999999  1.000114  0.999891  0.999984   0.999984   
3   2022-08-23  0.999983  1.000050  0.999886  0.999992   0.999992   
4   2022-08-24  0.999992  1.000105  0.999955  1.000039   1.000039   
5   2022-08-25  1.000034  1.000130  0.999959  1.000080   1.000080   
6   2022-08-26  1.000078  1.000139  0.999973  1.000043   1.000043   
7   2022-08-27  1.000056  1.000140  0.999971  1.000019   1.000019   
8   2022-08-28  1.000015  1.000087  0.999965  1.000028   1.000028   
9   2022-08-29  1.000051  1.000

[*********************100%***********************]  1 of 1 completed
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object
          Date      Open      High       Low     Close  Adj Close       Volume
0   2022-08-20  0.999954  1.000461  0.999737  1.000211   1.000211   7104084896
1   2022-08-21  1.000188  1.000397  0.999759  1.000141   1.000141   5446317941
2   2022-08-22  1.000114  1.000364  0.999764  1.000138   1.000138   5446317941
3   2022-08-23  1.000136  1.000395  0.999705  0.999982   0.999982   5446317941
4   2022-08-24  0.999999  1.000239  0.999647  0.999861   0.999861   7342467396
5   2022-08-25  0.999921  1.000195  0.999671  0.999960   0.999960   7247298408
6   2022-08-26  0.999979  1.000318  0.999549  0.999974   0.999974   9023777773
7   2022-08-27  0.999921  1.000350  0.999701  1.000009   1.000009  12367955268
8   2022-0

[*********************100%***********************]  1 of 1 completed
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object
          Date        Open        High         Low       Close   Adj Close  \
0   2022-08-20  279.620117  288.905518  278.332062  283.693146  283.693146   
1   2022-08-21  283.639099  303.657928  283.639099  302.056274  302.056274   
2   2022-08-22  302.034180  303.823792  289.306702  300.559113  300.559113   
3   2022-08-23  300.570923  301.195007  294.319672  299.029938  299.029938   
4   2022-08-24  299.015106  299.958679  294.712036  296.449677  296.449677   
5   2022-08-25  296.487305  306.138153  296.202087  301.583649  301.583649   
6   2022-08-26  301.556000  301.556000  277.851654  279.598175  279.598175   
7   2022-08-27  279.571838  283.850098  274.544373  277.801575  277.801575   
8   2022-08-28  277

...

[*********************100%***********************]  1 of 1 completed
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object
          Date          Open          High           Low         Close  \
0   2022-08-20  20861.605469  21313.988281  20824.750000  21158.873047   
1   2022-08-21  21153.751953  21651.466797  21096.291016  21552.857422   
2   2022-08-22  21553.335938  21553.335938  20916.958984  21388.128906   
3   2022-08-23  21388.078125  21635.585938  20938.185547  21547.212891   
4   2022-08-24  21550.443359  21766.447266  21183.322266  21415.431641   
5   2022-08-25  21414.544922  21763.105469  21393.933594  21594.441406   
6   2022-08-26  21594.535156  21778.048828  20184.121094  20215.691406   
7   2022-08-27  20219.498047  20304.832031  19862.253906  20045.621094   
8   2022-08-28  20044.380859  20117.212891  19564.11718

[*********************100%***********************]  1 of 1 completed
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object
         Date      Open      High       Low     Close  Adj Close     Volume
0  2022-08-20  6.907568  7.177405  6.772313  7.009903   7.009903  142278693
1  2022-08-21  7.008420  7.306613  6.987600  7.215944   7.215944  134765732
2  2022-08-22  7.214444  7.310630  6.679511  6.973576   6.973576  179017163
3  2022-08-23  6.975514  7.189586  6.784925  7.124180   7.124180  152762159
4  2022-08-24  7.121477  7.261781  6.891599  7.050521   7.050521  140714114
5  2022-08-25  7.050715  7.227865  6.947491  7.035765   7.035765  133546227
6  2022-08-26  7.035107  7.035107  6.118480  6.140219   6.140219  200332386
7  2022-08-27  6.140846  6.188772  5.979577  6.064946   6.064946  128446569
8  2022-08-28  6.063748  6.088385  5.

[*********************100%***********************]  1 of 1 completed
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object
          Date      Open      High       Low     Close  Adj Close   Volume
0   2022-08-20  5.186205  5.236299  5.016361  5.063648   5.063648  3100204
1   2022-08-21  5.062109  5.201651  5.022384  5.110690   5.110690  3320752
2   2022-08-22  5.110417  5.183166  5.015967  5.092703   5.092703  3207863
3   2022-08-23  5.094107  5.352363  5.090111  5.351367   5.351367  3461673
4   2022-08-24  5.350148  5.374066  5.220085  5.312732   5.312732  3444476
5   2022-08-25  5.311963  5.343719  5.131561  5.160338   5.160338  3600643
6   2022-08-26  5.160597  5.271091  5.025948  5.148439   5.148439  3441477
7   2022-08-27  5.146472  5.236131  5.087291  5.145066   5.145066  3530615
8   2022-08-28  5.146298  5.215632  5.146298  

[*********************100%***********************]  1 of 1 completed
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object
          Date       Open       High        Low      Close  Adj Close  \
0   2022-08-20  32.934219  34.504826  31.536892  32.852367  32.852367   
1   2022-08-21  32.837551  34.086895  32.279705  33.668453  33.668453   
2   2022-08-22  33.664742  34.010273  31.437956  34.010273  34.010273   
3   2022-08-23  34.015045  34.606216  32.382393  34.297916  34.297916   
4   2022-08-24  34.296329  36.629528  33.036243  35.309185  35.309185   
5   2022-08-25  35.312546  37.972614  35.312546  37.132900  37.132900   
6   2022-08-26  37.132378  38.154541  33.677547  33.887104  33.887104   
7   2022-08-27  33.890697  34.337322  32.201263  32.988548  32.988548   
8   2022-08-28  32.989780  33.336605  30.866867  30.870970  30.8

[*********************100%***********************]  1 of 1 completed
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object
          Date        Open        High        Low       Close   Adj Close  \
0   2022-08-20   54.024258   55.569393  53.154167   54.187553   54.187553   
1   2022-08-21   54.180012   55.935055  53.987640   55.492027   55.492027   
2   2022-08-22   55.497440   57.260742  52.519234   57.255676   57.255676   
3   2022-08-23   57.246269   57.581665  54.280712   57.079433   57.079433   
4   2022-08-24   57.077991   57.463272  55.827904   56.478168   56.478168   
5   2022-08-25   56.475605   57.368172  56.088352   56.792576   56.792576   
6   2022-08-26   56.782242   57.959911  52.514240   52.744450   52.744450   
7   2022-08-27   52.758629   53.282288  52.116840   53.050240   53.050240   
8   2022-08-28   53.047722  

In [42]:
database_info(c, 'ETH','Date')

O banco de dados contém as seguintes tabelas: 


BTC
ETH
DOT
ADA
USDT
USDC
BNB
BUSD
XRP
SOL
DOGE
HEX
SHIB
DAI
WTRX
AVAX
MATIC
TRX
STETH
WBTC
UNI1
LEO
ETC
LTC


O nome da tabela solicitada é :ETH


O nome das colunas são: 


['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']


O número total de linhas é: 2036


A data mais recente dos dados é: 2023-06-06 00:00:00


Todas as datas distintas e quantas linhas cada uma possui são: 


('2017-11-09 00:00:00', 1)
('2017-11-10 00:00:00', 1)
('2017-11-11 00:00:00', 1)
('2017-11-12 00:00:00', 1)
('2017-11-13 00:00:00', 1)
('2017-11-14 00:00:00', 1)
('2017-11-15 00:00:00', 1)
('2017-11-16 00:00:00', 1)
('2017-11-17 00:00:00', 1)
('2017-11-18 00:00:00', 1)
('2017-11-19 00:00:00', 1)
('2017-11-20 00:00:00', 1)
('2017-11-21 00:00:00', 1)
('2017-11-22 00:00:00', 1)
('2017-11-23 00:00:00', 1)
('2017-11-24 00:00:00', 1)
('2017-11-25 00:00:00', 1)
('2017-11-26 00:00:00', 1)
('2017-11-27 00:00:00', 1)
('2017-11-28 00:00:00', 1)
('2017-11-29 0

In [ ]:
def plot(title,crypto, conn, startDate, endDate):
    
    
    df=df_from_sql_query(crypto, "Date", conn, startDate, endDate)
    
    df['Date']= pd.to_datetime(df['Date'])
    
    df=df.drop(columns=['Adj Close'])
    
#     eth_df.set_index('Date')

    fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'])])

    fig.update_layout(xaxis_rangeslider_visible=False)

    fig.update_layout(
        title= title,
        yaxis_title='Price',

    )

    fig.write_html(title + ".html")
    fig.write_html(title + ".html")

    fig.show()

In [ ]:
plot('ETHER-USD','ETH', conn, '2022-08-01', '2022-09-14')

In [ ]:
plot('BTC-USD','BTC', conn, '2022-08-01', '2022-09-14')